# Import dependencies

In [1]:
import os
import sys

sys.path.insert(0, os.path.dirname(os.getcwd())) 

In [2]:
import time
import gc
import json

import numpy as np
import pandas as pd

from transformers import BertTokenizer, BertModel
from transformers import logging

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

from InputDataset import InputDataset

import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence

from torch import cuda

from absa_models.ABSA_BERT_Dropout_BiLSTM_Linear import ABSA_BERT_Dropout_BiLSTM_Linear

In [3]:
device = 'cuda' if cuda.is_available() else 'cpu'
logging.set_verbosity_error() 

In [4]:
print(torch.cuda.get_device_name(0))
print(f"Memory: {torch.cuda.get_device_properties(0).total_memory // 1024 ** 3} GB")

NVIDIA GeForce RTX 2060 SUPER
Memory: 8 GB


In [5]:
def clear_memory():
    torch.cuda.empty_cache()

    with torch.no_grad():
        torch.cuda.empty_cache()

    gc.collect()

# Load Data

In [6]:
DATASET = 'ABSA_SemEval16_Restaurants_train.json'

In [7]:
df = pd.json_normalize(json.load(open(DATASET)))

In [8]:
df.head()

,text,tokens,absa_tags
0,Judging from previous posts this used to be a ...,"[Judging, from, previous, posts, this, used, t...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
1,"We, there were four of us, arrived at noon - t...","[We, ,, there, were, four, of, us, ,, arrived,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"They never brought us complimentary noodles, i...","[They, never, brought, us, complimentary, nood...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,The food was lousy - too sweet or too salty an...,"[The, food, was, lousy, -, too, sweet, or, too...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"
4,The food was lousy - too sweet or too salty an...,"[The, food, was, lousy, -, too, sweet, or, too...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"


# Train & Validate

In [9]:
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4

EPOCHS = 2

LEARNING_RATE = 1e-5

TRAIN_SPLIT = 0.8

NO_RUNS = 10

In [10]:
BERT_FINE_TUNED_PATH = '../../../results/ABSA/SemEval16 - Task 5 - Restaurants/models/bert_fine_tuned.pth'

In [11]:
MODEL_OUTPUT = '../../../results/ABSA/SemEval16 - Task 5 - Restaurants/models/bert_fine_tuned_dropout_bilstm_linear.pth'
STATS_OUTPUT = '../../../results/ABSA/SemEval16 - Task 5 - Restaurants/stats/bert_fine_tuned_dropout_bilstm_linear.csv'

In [12]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [13]:
best_accuracy = 0.0

In [14]:
def create_mini_batch(samples):
    ids_tensors = [s[1] for s in samples]
    ids_tensors = pad_sequence(ids_tensors, batch_first=True).to(device)

    tags_tensors = [s[2] for s in samples]
    tags_tensors = pad_sequence(tags_tensors, batch_first=True).to(device)
    
    masks_tensors = torch.zeros(ids_tensors.shape, dtype=torch.long).to(device)
    masks_tensors = masks_tensors.masked_fill(ids_tensors != 0, 1).to(device)
    
    return ids_tensors, tags_tensors, masks_tensors

In [15]:
def train(epoch, model, loss_fn, optimizer, dataloader):
    model.train()

    dataloader_len = len(dataloader)

    for _,data in enumerate(dataloader, 0):
        optimizer.zero_grad()

        ids_tensors, tags_tensors, masks_tensors = data

        outputs = model(ids_tensors, masks_tensors)

        loss = loss_fn(outputs.view(-1, 4), tags_tensors.view(-1))
        
        if _ % (dataloader_len // 10) == 0:
            print(f"Epoch: {epoch}/{EPOCHS}, Batch: {_}/{dataloader_len}, Loss: {loss.item()}")
        
        loss.backward()
        
        optimizer.step()

In [16]:
def validation(model, dataloader):
    model.eval()
    
    fin_targets=[]
    fin_outputs=[]

    with torch.no_grad():
        for _, data in enumerate(dataloader, 0):
            ids_tensors, tags_tensors, masks_tensors = data
            ids_tensors = ids_tensors.to(device)
            tags_tensors = tags_tensors.to(device)
            masks_tensors = masks_tensors.to(device)

            outputs = model(ids_tensors, masks_tensors)
            
            _, predictions = torch.max(outputs, dim=2)

            fin_outputs += list([int(p) for pred in predictions for p in pred])
            fin_targets += list([int(tag) for tags_tensor in tags_tensors for tag in tags_tensor])

    return fin_outputs, fin_targets

In [17]:
results = pd.DataFrame(columns=['accuracy','precision_score_micro','precision_score_macro','recall_score_micro','recall_score_macro','f1_score_micro','f1_score_macro', 'execution_time'])

In [18]:
for i in range(NO_RUNS):
    # clear cache cuda
    torch.cuda.empty_cache()
    with torch.no_grad():
        torch.cuda.empty_cache()
    gc.collect()

    start_time = time.time()

    print(f"Run {i + 1}/{NO_RUNS}")

    train_dataset = df.sample(frac=TRAIN_SPLIT)
    test_dataset = df.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    training_set = InputDataset(train_dataset, tokenizer)
    testing_set = InputDataset(test_dataset, tokenizer)

    train_dataloader = DataLoader(
        training_set,
        sampler = RandomSampler(train_dataset),
        batch_size = TRAIN_BATCH_SIZE,
        drop_last = True,
        collate_fn=create_mini_batch
    )

    validation_dataloader = DataLoader(
        testing_set,
        sampler = SequentialSampler(testing_set),
        batch_size = VALID_BATCH_SIZE,
        drop_last = True,
        collate_fn=create_mini_batch
    )

    model = ABSA_BERT_Dropout_BiLSTM_Linear(torch.load(BERT_FINE_TUNED_PATH), dropout=0.3, bilstm_in_features=256, no_out_labels=4, device=device).to(device)

    optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE)
    loss_fn = torch.nn.CrossEntropyLoss()

    for epoch in range(EPOCHS):
        train(epoch, model, loss_fn, optimizer, train_dataloader)

    outputs, targets = validation(model, validation_dataloader)
    
    accuracy = accuracy_score(targets, outputs)
    precision_score_micro = precision_score(targets, outputs, average='micro')
    precision_score_macro = precision_score(targets, outputs, average='macro')
    recall_score_micro = recall_score(targets, outputs, average='micro')
    recall_score_macro = recall_score(targets, outputs, average='macro')
    f1_score_micro = f1_score(targets, outputs, average='micro')
    f1_score_macro = f1_score(targets, outputs, average='macro')

    execution_time = time.time() - start_time

    results.loc[i] = [accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro, execution_time]

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model, MODEL_OUTPUT)

    del train_dataset
    del test_dataset
    del training_set
    del testing_set
    del model
    del loss_fn
    del optimizer
    del outputs
    del targets

Run 1/10
Epoch: 0/2, Batch: 0/501, Loss: 1.4245624542236328
Epoch: 0/2, Batch: 50/501, Loss: 0.3275899887084961
Epoch: 0/2, Batch: 100/501, Loss: 0.15733234584331512
Epoch: 0/2, Batch: 150/501, Loss: 0.1870076060295105
Epoch: 0/2, Batch: 200/501, Loss: 0.14329196512699127
Epoch: 0/2, Batch: 250/501, Loss: 0.14260898530483246
Epoch: 0/2, Batch: 300/501, Loss: 0.06159212067723274
Epoch: 0/2, Batch: 350/501, Loss: 0.12087592482566833
Epoch: 0/2, Batch: 400/501, Loss: 0.1780386120080948
Epoch: 0/2, Batch: 450/501, Loss: 0.14162801206111908
Epoch: 0/2, Batch: 500/501, Loss: 0.0847201943397522
Epoch: 1/2, Batch: 0/501, Loss: 0.08411674201488495
Epoch: 1/2, Batch: 50/501, Loss: 0.05335589125752449
Epoch: 1/2, Batch: 100/501, Loss: 0.07639379054307938
Epoch: 1/2, Batch: 150/501, Loss: 0.06630731374025345
Epoch: 1/2, Batch: 200/501, Loss: 0.11510130763053894
Epoch: 1/2, Batch: 250/501, Loss: 0.1399868279695511
Epoch: 1/2, Batch: 300/501, Loss: 0.10587586462497711
Epoch: 1/2, Batch: 350/501, Los

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 2/10
Epoch: 0/2, Batch: 0/501, Loss: 1.4087523221969604
Epoch: 0/2, Batch: 50/501, Loss: 0.3260759115219116
Epoch: 0/2, Batch: 100/501, Loss: 0.11777027696371078
Epoch: 0/2, Batch: 150/501, Loss: 0.13564379513263702
Epoch: 0/2, Batch: 200/501, Loss: 0.23264552652835846
Epoch: 0/2, Batch: 250/501, Loss: 0.13371409475803375
Epoch: 0/2, Batch: 300/501, Loss: 0.11931739002466202
Epoch: 0/2, Batch: 350/501, Loss: 0.11925467848777771
Epoch: 0/2, Batch: 400/501, Loss: 0.06234820932149887
Epoch: 0/2, Batch: 450/501, Loss: 0.17038434743881226
Epoch: 0/2, Batch: 500/501, Loss: 0.09486900269985199
Epoch: 1/2, Batch: 0/501, Loss: 0.2564319670200348
Epoch: 1/2, Batch: 50/501, Loss: 0.0758480355143547
Epoch: 1/2, Batch: 100/501, Loss: 0.03141826018691063
Epoch: 1/2, Batch: 150/501, Loss: 0.03875114023685455
Epoch: 1/2, Batch: 200/501, Loss: 0.10888990759849548
Epoch: 1/2, Batch: 250/501, Loss: 0.03600764647126198
Epoch: 1/2, Batch: 300/501, Loss: 0.042213860899209976
Epoch: 1/2, Batch: 350/501, 

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 3/10
Epoch: 0/2, Batch: 0/501, Loss: 1.5532532930374146
Epoch: 0/2, Batch: 50/501, Loss: 0.4134198725223541
Epoch: 0/2, Batch: 100/501, Loss: 0.21246513724327087
Epoch: 0/2, Batch: 150/501, Loss: 0.0608782060444355
Epoch: 0/2, Batch: 200/501, Loss: 0.2729940116405487
Epoch: 0/2, Batch: 250/501, Loss: 0.1138898953795433
Epoch: 0/2, Batch: 300/501, Loss: 0.14737197756767273
Epoch: 0/2, Batch: 350/501, Loss: 0.18809998035430908
Epoch: 0/2, Batch: 400/501, Loss: 0.08037018030881882
Epoch: 0/2, Batch: 450/501, Loss: 0.06574970483779907
Epoch: 0/2, Batch: 500/501, Loss: 0.12902379035949707
Epoch: 1/2, Batch: 0/501, Loss: 0.06655297428369522
Epoch: 1/2, Batch: 50/501, Loss: 0.12400758266448975
Epoch: 1/2, Batch: 100/501, Loss: 0.1167212501168251
Epoch: 1/2, Batch: 150/501, Loss: 0.09341291338205338
Epoch: 1/2, Batch: 200/501, Loss: 0.07993372529745102
Epoch: 1/2, Batch: 250/501, Loss: 0.04226921126246452
Epoch: 1/2, Batch: 300/501, Loss: 0.010010933503508568
Epoch: 1/2, Batch: 350/501, Lo

...

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 9/10
Epoch: 0/2, Batch: 0/501, Loss: 1.3969430923461914
Epoch: 0/2, Batch: 50/501, Loss: 0.29511919617652893
Epoch: 0/2, Batch: 100/501, Loss: 0.16974599659442902
Epoch: 0/2, Batch: 150/501, Loss: 0.07268745452165604
Epoch: 0/2, Batch: 200/501, Loss: 0.10692350566387177
Epoch: 0/2, Batch: 250/501, Loss: 0.13919739425182343
Epoch: 0/2, Batch: 300/501, Loss: 0.14729122817516327
Epoch: 0/2, Batch: 350/501, Loss: 0.15302777290344238
Epoch: 0/2, Batch: 400/501, Loss: 0.04983145371079445
Epoch: 0/2, Batch: 450/501, Loss: 0.0906340628862381
Epoch: 0/2, Batch: 500/501, Loss: 0.08666247129440308
Epoch: 1/2, Batch: 0/501, Loss: 0.07242336124181747
Epoch: 1/2, Batch: 50/501, Loss: 0.020649464800953865
Epoch: 1/2, Batch: 100/501, Loss: 0.021694522351026535
Epoch: 1/2, Batch: 150/501, Loss: 0.030912764370441437
Epoch: 1/2, Batch: 200/501, Loss: 0.05036359280347824
Epoch: 1/2, Batch: 250/501, Loss: 0.09514054656028748
Epoch: 1/2, Batch: 300/501, Loss: 0.10801661759614944
Epoch: 1/2, Batch: 350/5

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 10/10
Epoch: 0/2, Batch: 0/501, Loss: 1.4566614627838135
Epoch: 0/2, Batch: 50/501, Loss: 0.24486267566680908
Epoch: 0/2, Batch: 100/501, Loss: 0.15688090026378632
Epoch: 0/2, Batch: 150/501, Loss: 0.21815237402915955
Epoch: 0/2, Batch: 200/501, Loss: 0.10492637008428574
Epoch: 0/2, Batch: 250/501, Loss: 0.09181434661149979
Epoch: 0/2, Batch: 300/501, Loss: 0.06927237659692764
Epoch: 0/2, Batch: 350/501, Loss: 0.05551331862807274
Epoch: 0/2, Batch: 400/501, Loss: 0.07030792534351349
Epoch: 0/2, Batch: 450/501, Loss: 0.2451610118150711
Epoch: 0/2, Batch: 500/501, Loss: 0.17908231914043427
Epoch: 1/2, Batch: 0/501, Loss: 0.06461773812770844
Epoch: 1/2, Batch: 50/501, Loss: 0.06869976222515106
Epoch: 1/2, Batch: 100/501, Loss: 0.06355331838130951
Epoch: 1/2, Batch: 150/501, Loss: 0.0119858393445611
Epoch: 1/2, Batch: 200/501, Loss: 0.02232171781361103
Epoch: 1/2, Batch: 250/501, Loss: 0.05062120780348778
Epoch: 1/2, Batch: 300/501, Loss: 0.04179224371910095
Epoch: 1/2, Batch: 350/501,

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
results

,accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro,execution_time
0,0.985860,0.985860,0.680909,0.985860,0.688010,0.985860,0.683279,129.921999
1,0.978756,0.978756,0.611569,0.978756,0.694209,0.978756,0.646792,117.928999
2,0.984760,0.984760,0.673924,0.984760,0.694054,0.984760,0.683737,117.782001
3,0.986065,0.986065,0.676308,0.986065,0.676815,0.986065,0.676465,116.665001
4,0.986077,0.986077,0.660187,0.986077,0.698459,0.986077,0.677305,117.215999
5,0.984499,0.984499,0.659566,0.984499,0.685239,0.984499,0.671911,117.351001
6,0.982896,0.982896,0.661383,0.982896,0.713460,0.982896,0.685443,117.412999
7,0.986366,0.986366,0.682138,0.986366,0.702056,0.986366,0.691748,116.962996
8,0.981915,0.981915,0.642603,0.981915,0.693438,0.981915,0.666296,117.267000
9,0.979707,0.979707,0.615907,0.979707,0.708705,0.979707,0.654890,117.081000


In [20]:
results.to_csv(STATS_OUTPUT)